In [1]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np
import requests
import folium
from folium.plugins import MarkerCluster
import re


In [21]:
oldman=pd.read_csv('./12_21_oldman (1).csv')
oldman.head()

,사고다발지FID,사고다발지ID,법정동코드,지점코드,시도시군구명,지점명,사고건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,경도,위도,다발지역폴리곤
0,145935,2013098,1111018000,11110001,서울특별시 종로구1,서울특별시 종로구 교북동(독립문_의주로_진출_1 부근),3,3,0,3,0,0,126.960630,37.571937,"{""type"":""Polygon"",""coordinates"":[[[126.9622309..."
1,144334,2013098,1111016700,11110002,서울특별시 종로구2,서울특별시 종로구 충신동(이화사거리 부근),5,5,0,5,0,0,127.004403,37.576104,"{""type"":""Polygon"",""coordinates"":[[[127.0060035..."
2,151689,2013098,1111015400,11110003,서울특별시 종로구3,서울특별시 종로구 장사동(종묘공원 부근),6,6,0,4,1,1,126.993647,37.570706,"{""type"":""Polygon"",""coordinates"":[[[126.9952475..."
3,151688,2013098,1111017400,11110004,서울특별시 종로구4,서울특별시 종로구 창신동(동묘앞역_다산로_진입 부근),6,6,1,5,0,0,127.016856,37.572359,"{""type"":""Polygon"",""coordinates"":[[[127.0184565..."
4,144645,2013098,1114016000,11140001,서울특별시 중구1,서울특별시 중구 인현동1가(대림상가앞 부근),3,3,0,1,2,0,126.995342,37.566380,"{""type"":""Polygon"",""coordinates"":[[[126.9969423..."


In [22]:
oldman=oldman[oldman['시도시군구명'].str.contains('천안시')]
oldman.drop('사고다발지FID',axis=1,inplace=True)
oldman.drop('사고다발지ID',axis=1,inplace=True)
oldman.drop('법정동코드',axis=1,inplace=True)
oldman.drop('지점코드',axis=1,inplace=True)
oldman

,시도시군구명,지점명,사고건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,경도,위도,다발지역폴리곤
384,충청남도 천안시 동남구1,충청남도 천안시 동남구 원성동((자연피아노학원) 부근),3,3,0,3,0,0,127.158540,36.803724,"{""type"":""Polygon"",""coordinates"":[[[127.1601246..."
385,충청남도 천안시 동남구2,충청남도 천안시 동남구 원성동((르노삼성써비스센터) 부근),3,3,1,1,1,0,127.159210,36.799451,"{""type"":""Polygon"",""coordinates"":[[[127.1607947..."
386,충청남도 천안시 동남구3,충청남도 천안시 동남구 대흥동((학하호도과자) 부근),4,4,0,1,3,0,127.148832,36.809691,"{""type"":""Polygon"",""coordinates"":[[[127.1504165..."
387,충청남도 천안시 서북구1,충청남도 천안시 서북구 쌍용동((동부빌라1동) 부근),5,6,2,2,2,0,127.132786,36.798132,"{""type"":""Polygon"",""coordinates"":[[[127.1343711..."
909,충청남도 천안시 동남구1,충청남도 천안시 동남구 신부동((민현빌딩) 부근),4,4,1,3,0,0,127.155127,36.818762,"{""type"":""Polygon"",""coordinates"":[[[127.1567116..."
910,충청남도 천안시 동남구2,충청남도 천안시 동남구 영성동((국산빌딩) 부근),4,4,0,2,2,0,127.149309,36.800316,"{""type"":""Polygon"",""coordinates"":[[[127.1508934..."
911,충청남도 천안시 동남구3,충청남도 천안시 동남구 대흥동((학하호도과자) 부근),3,3,0,2,1,0,127.148152,36.809360,"{""type"":""Polygon"",""coordinates"":[[[127.1497373..."
912,충청남도 천안시 서북구1,충청남도 천안시 서북구 성환읍(성환읍사무소입구 부근),3,3,1,1,1,0,127.135424,36.918911,"{""type"":""Polygon"",""coordinates"":[[[127.1370113..."
1526,충청남도 천안시 동남구1,충청남도 천안시 동남구 영성동((영진약국) 부근),9,10,0,5,5,0,127.150534,36.803235,"{""type"":""Polygon"",""coordinates"":[[[127.1521184..."
2103,충청남도 천안시 동남구1,충청남도 천안시 동남구 대흥동((서울꽃화원) 부근),6,6,0,5,1,0,127.148504,36.810553,"{""type"":""Polygon"",""coordinates"":[[[127.1503011..."


In [24]:
# 네이버 API key 가져오기
def getAPIkey():
    f=open('./secret_key.txt',"r")
    return f.read()

In [60]:
CLIENT_ID = 'vjgu0f53rm'
key=getAPIkey()
CLIENT_SECRET = key


# 경도 위도 추출
lenght=len(oldman)
loca=[]
for i in range(0,lenght):
    x=oldman.iloc[i]['경도']
    y=oldman.iloc[i]['위도']
    loca.append([x,y])


amd_name=[]
for i in loca:
	x,y=i
	coords_param=f'{x},{y}'
	headers = {'X-NCP-APIGW-API-KEY-ID': CLIENT_ID, 'X-NCP-APIGW-API-KEY': CLIENT_SECRET}
	url2=f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={coords_param}&orders={'admcode,legalcode'}&output={'json'}"
	response2 = requests.get(url2, headers=headers)
	response_data2=response2.json()
	data1=response_data2['results'][0]['region']['area2']['name']
	data2=response_data2['results'][0]['region']['area3']['name']
	data1=data1.replace(' ','')
	amd_name.append(data1+" "+data2)
        
oldman['지역']=amd_name
oldman

,시도시군구명,지점명,사고건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,경도,위도,지역,읍면동,지도
384,충청남도 천안시 동남구1,충청남도 천안시 동남구 원성동((자연피아노학원) 부근),3,3,0,3,0,0,127.158540,36.803724,천안시동남구 원성2동,천안시동남구 원성동,"""type"":""Polygon"",""coordinates"":[[[127.1601246,..."
385,충청남도 천안시 동남구2,충청남도 천안시 동남구 원성동((르노삼성써비스센터) 부근),3,3,1,1,1,0,127.159210,36.799451,천안시동남구 원성2동,천안시동남구 원성동,"""type"":""Polygon"",""coordinates"":[[[127.1607947,..."
386,충청남도 천안시 동남구3,충청남도 천안시 동남구 대흥동((학하호도과자) 부근),4,4,0,1,3,0,127.148832,36.809691,천안시동남구 중앙동,천안시동남구 중앙동,"""type"":""Polygon"",""coordinates"":[[[127.1504165,..."
387,충청남도 천안시 서북구1,충청남도 천안시 서북구 쌍용동((동부빌라1동) 부근),5,6,2,2,2,0,127.132786,36.798132,천안시서북구 쌍용1동,천안시서북구 쌍용동,"""type"":""Polygon"",""coordinates"":[[[127.1343711,..."
909,충청남도 천안시 동남구1,충청남도 천안시 동남구 신부동((민현빌딩) 부근),4,4,1,3,0,0,127.155127,36.818762,천안시동남구 신안동,천안시동남구 신안동,"""type"":""Polygon"",""coordinates"":[[[127.1567116,..."
910,충청남도 천안시 동남구2,충청남도 천안시 동남구 영성동((국산빌딩) 부근),4,4,0,2,2,0,127.149309,36.800316,천안시동남구 중앙동,천안시동남구 중앙동,"""type"":""Polygon"",""coordinates"":[[[127.1508934,..."
911,충청남도 천안시 동남구3,충청남도 천안시 동남구 대흥동((학하호도과자) 부근),3,3,0,2,1,0,127.148152,36.809360,천안시동남구 중앙동,천안시동남구 중앙동,"""type"":""Polygon"",""coordinates"":[[[127.1497373,..."
912,충청남도 천안시 서북구1,충청남도 천안시 서북구 성환읍(성환읍사무소입구 부근),3,3,1,1,1,0,127.135424,36.918911,천안시서북구 성환읍,천안시서북구 성환읍,"""type"":""Polygon"",""coordinates"":[[[127.1370113,..."
1526,충청남도 천안시 동남구1,충청남도 천안시 동남구 영성동((영진약국) 부근),9,10,0,5,5,0,127.150534,36.803235,천안시동남구 중앙동,천안시동남구 중앙동,"""type"":""Polygon"",""coordinates"":[[[127.1521184,..."
2103,충청남도 천안시 동남구1,충청남도 천안시 동남구 대흥동((서울꽃화원) 부근),6,6,0,5,1,0,127.148504,36.810553,천안시동남구 중앙동,천안시동남구 중앙동,"""type"":""Polygon"",""coordinates"":[[[127.15030112..."


In [59]:
cleaned_data = [re.sub(r'\d+', '', item) for item in amd_name]
clean_reg=[re.sub(r'^{|}$', '', item) for item in oldman['다발지역폴리곤']]

print(clean_reg)
a,b,c=clean_reg[0].split(',')
oldman['지도']=clean_reg
oldman.drop('다발지역폴리곤',axis=1,inplace=True)
oldman

KeyError: '다발지역폴리곤'

In [63]:
oldman

,시도시군구명,지점명,사고건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,경도,위도,지역,읍면동,지도,Name_dict
384,충청남도 천안시 동남구1,충청남도 천안시 동남구 원성동((자연피아노학원) 부근),3,3,0,3,0,0,127.158540,36.803724,천안시동남구 원성2동,천안시동남구 원성동,"""type"":""Polygon"",""coordinates"":[[[127.1601246,...","{""type"":""Polygon"",""coordinates"":[[[127.1601246..."
385,충청남도 천안시 동남구2,충청남도 천안시 동남구 원성동((르노삼성써비스센터) 부근),3,3,1,1,1,0,127.159210,36.799451,천안시동남구 원성2동,천안시동남구 원성동,"""type"":""Polygon"",""coordinates"":[[[127.1607947,...","{""type"":""Polygon"",""coordinates"":[[[127.1607947..."
386,충청남도 천안시 동남구3,충청남도 천안시 동남구 대흥동((학하호도과자) 부근),4,4,0,1,3,0,127.148832,36.809691,천안시동남구 중앙동,천안시동남구 중앙동,"""type"":""Polygon"",""coordinates"":[[[127.1504165,...","{""type"":""Polygon"",""coordinates"":[[[127.1504165..."
387,충청남도 천안시 서북구1,충청남도 천안시 서북구 쌍용동((동부빌라1동) 부근),5,6,2,2,2,0,127.132786,36.798132,천안시서북구 쌍용1동,천안시서북구 쌍용동,"""type"":""Polygon"",""coordinates"":[[[127.1343711,...","{""type"":""Polygon"",""coordinates"":[[[127.1343711..."
909,충청남도 천안시 동남구1,충청남도 천안시 동남구 신부동((민현빌딩) 부근),4,4,1,3,0,0,127.155127,36.818762,천안시동남구 신안동,천안시동남구 신안동,"""type"":""Polygon"",""coordinates"":[[[127.1567116,...","{""type"":""Polygon"",""coordinates"":[[[127.1567116..."
910,충청남도 천안시 동남구2,충청남도 천안시 동남구 영성동((국산빌딩) 부근),4,4,0,2,2,0,127.149309,36.800316,천안시동남구 중앙동,천안시동남구 중앙동,"""type"":""Polygon"",""coordinates"":[[[127.1508934,...","{""type"":""Polygon"",""coordinates"":[[[127.1508934..."
911,충청남도 천안시 동남구3,충청남도 천안시 동남구 대흥동((학하호도과자) 부근),3,3,0,2,1,0,127.148152,36.809360,천안시동남구 중앙동,천안시동남구 중앙동,"""type"":""Polygon"",""coordinates"":[[[127.1497373,...","{""type"":""Polygon"",""coordinates"":[[[127.1497373..."
912,충청남도 천안시 서북구1,충청남도 천안시 서북구 성환읍(성환읍사무소입구 부근),3,3,1,1,1,0,127.135424,36.918911,천안시서북구 성환읍,천안시서북구 성환읍,"""type"":""Polygon"",""coordinates"":[[[127.1370113,...","{""type"":""Polygon"",""coordinates"":[[[127.1370113..."
1526,충청남도 천안시 동남구1,충청남도 천안시 동남구 영성동((영진약국) 부근),9,10,0,5,5,0,127.150534,36.803235,천안시동남구 중앙동,천안시동남구 중앙동,"""type"":""Polygon"",""coordinates"":[[[127.1521184,...","{""type"":""Polygon"",""coordinates"":[[[127.1521184..."
2103,충청남도 천안시 동남구1,충청남도 천안시 동남구 대흥동((서울꽃화원) 부근),6,6,0,5,1,0,127.148504,36.810553,천안시동남구 중앙동,천안시동남구 중앙동,"""type"":""Polygon"",""coordinates"":[[[127.15030112...","{""type"":""Polygon"",""coordinates"":[[[127.1503011..."


In [27]:
oldman['읍면동']=cleaned_data

In [28]:
oldman.to_excel('노인 사망자 데이터.xlsx', index=False)  # Set index=False to exclude row numbers

In [64]:
# cheonan_features = [feature for feature in oldman['features'] if '천안시' in feature['properties']['adm_nm']]

'''
"type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {
                "adm_nm": "\ucda9\uccad\ub0a8\ub3c4 \ucc9c\uc548\uc2dc\ub3d9\ub0a8\uad6c \ubaa9\ucc9c\uc74d",
                "adm_cd": "3401111",
                "adm_cd2": "4413125000",
                "sgg": "44131",
                "sido": "44",
                "sidonm": "\ucda9\uccad\ub0a8\ub3c4",
                "temp": "\ucc9c\uc548\uc2dc\ub3d9\ub0a8\uad6c \ubaa9\ucc9c\uc74d",
                "sggnm": "\ucc9c\uc548\uc2dc\ub3d9\ub0a8\uad6c",
                "adm_cd8": "34011110"
            },
'''

col=[]
for index,row in oldman.iterrows():
    onedic={"type":'feature'}
    onedic['properties']={"name":row['지점명']}
    onedic['geometry']=row['Name_dict']
    col.append(onedic)


cheonan_json = {
    "type": "FeatureCollection",
    "features": col
}

print(cheonan_json)

{'type': 'FeatureCollection', 'features': [{'type': 'feature', 'properties': {'name': '충청남도 천안시 동남구 원성동((자연피아노학원) 부근)'}, 'geometry': {'"type":"Polygon","coordinates":[[[127.1601246,36.8049983],[127.1597849,36.8052224],[127.1593975,36.8053889],[127.158977,36.8054915],[127.1585398,36.8055261],[127.1581026,36.8054915],[127.1576821,36.8053889],[127.1572947,36.8052224],[127.156955,36.8049983],[127.1566763,36.8047252],[127.1564692,36.8044136],[127.1563417,36.8040755],[127.1562987,36.8037239],[127.1563417,36.8033723],[127.1564693,36.8030342],[127.1566764,36.8027226],[127.1569551,36.8024495],[127.1572947,36.8022254],[127.1576822,36.8020589],[127.1581026,36.8019563],[127.1585398,36.8019217],[127.158977,36.8019563],[127.1593974,36.8020589],[127.1597849,36.8022254],[127.1601245,36.8024495],[127.1604032,36.8027226],[127.1606103,36.8030342],[127.1607379,36.8033723],[127.160781,36.8037239],[127.1607379,36.8040755],[127.1606104,36.8044136],[127.1604033,36.8047252],[127.1601246,36.8049983]]]'}}, {'typ

In [17]:
api_key='Awo%2B6chUiLHMtn6rxDbvq9Pp3ufImIe0%2Bm0w806qv56aTmVyHDt1gXqXTM7aAB0h'


for i in range(2016,2019):
	url2=f"http://taas.koroad.or.kr/data/rest/stt?authKey={api_key}&sido=1600&searchYearCd={i}&guGun=1602&type=json"
	response2 = requests.get(url2)
	response_data2=response2.json()
	a=pd.DataFrame(response_data2)
	print(a)
	break

     resultCode    resultMsg  \
item         00  NORMAL_CODE   

                                                  items  totalCount  \
item  [{'std_year': '2016', 'sido_sgg_nm': '충청남도 천안시...          12   

      numOfRows  pageNo  
item         12       1  


In [67]:
import pandas as pd
